将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

我们可以用两种方式来试着做一做，我们试一试用k相邻和支持向量机两种咱们最熟悉也比较简单的方法来试一试， 其中空缺值咱们使用众数填充。

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

raw_data = pd.read_csv('data//fraudulent.csv')
tmp = [0, 0]
for j in raw_data.columns:
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] == 0 or raw_data.loc[i, j] == 1:
            tmp[int(raw_data.loc[i, j])] += 1
    max_tmp = 0
    if tmp[0] < tmp[1]:
        max_tmp = 1
    tmp = [0, 0]
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] != 0 and raw_data.loc[i, j] != 1:
            raw_data.loc[i, j] = max_tmp
y = raw_data['y']
X = raw_data[[_ for _ in raw_data.columns if _ != 'y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state = 1)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

y_predict = knn.predict(X_test)
f1 = f1_score(y_test, y_predict)
print(f'KNN f1 = {f1:.2f}')

svm_classifier = svm.SVC(kernel = 'linear', C = 1, gamma = 'scale')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
_f1 = f1_score(y_test, y_pred)
print(f'SVM f1 = {_f1:.2f}')

KNN f1 = 0.82
SVM f1 = 0.85


knn算法的f1值为0.82，而SVM达到0.85，并不太理想，我们想办法进行优化。对于数据预处理的方面，众数填充可能引入偏差。对于缺失值较多的数据我们选择删除。于是我们采取策略，如果一行的空缺值有三个及以上，这行数据就要删除掉。

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

raw_data = pd.read_csv('data//fraudulent.csv')
# print(raw_data.shape[0])

for i in range(raw_data.shape[0]):
    w_num = 0
    for j in raw_data.columns:
        if raw_data.loc[i, j] != raw_data.loc[i, j]:
            w_num += 1
    if w_num >= 3:
        raw_data.drop(index = i, inplace = True)
raw_data.reset_index(drop = True, inplace = True)
# print(raw_data.shape[0])

tmp = [0, 0]
for j in raw_data.columns:
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] == 0 or raw_data.loc[i, j] == 1:
            tmp[int(raw_data.loc[i, j])] += 1
    max_tmp = 0
    if tmp[0] < tmp[1]:
        max_tmp = 1
    tmp = [0, 0]
    # print(max_tmp, end = ' ')
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] != 0 and raw_data.loc[i, j] != 1:
            raw_data.loc[i, j] = max_tmp
y = raw_data['y']
X = raw_data[[_ for _ in raw_data.columns if _ != 'y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state = 1)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

y_predict = knn.predict(X_test)
f1 = f1_score(y_test, y_predict)
print(f'KNN f1 = {f1:.2f}')

svm_classifier = svm.SVC(kernel = 'linear', C = 1, gamma = 'scale')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
_f1 = f1_score(y_test, y_pred)
print(f'SVM f1 = {_f1:.2f}')

KNN f1 = 0.87
SVM f1 = 0.84


删除了一些数据之后，KNN算法的f1值提升到了0.87，而SVM算法却降低了。我们尝试进行PCA降维。

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

raw_data = pd.read_csv('data//fraudulent.csv')
# print(raw_data.shape[0])

for i in range(raw_data.shape[0]):
    w_num = 0
    for j in raw_data.columns:
        if raw_data.loc[i, j] != raw_data.loc[i, j]:
            w_num += 1
    if w_num >= 3:
        raw_data.drop(index = i, inplace = True)
raw_data.reset_index(drop = True, inplace = True)
# print(raw_data.shape[0])

tmp = [0, 0]
for j in raw_data.columns:
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] == 0 or raw_data.loc[i, j] == 1:
            tmp[int(raw_data.loc[i, j])] += 1
    max_tmp = 0
    if tmp[0] < tmp[1]:
        max_tmp = 1
    tmp = [0, 0]
    # print(max_tmp, end = ' ')
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] != 0 and raw_data.loc[i, j] != 1:
            raw_data.loc[i, j] = max_tmp


y = raw_data['y']
X = raw_data[[_ for _ in raw_data.columns if _ != 'y']]
pca = PCA(n_components = 2)
X = pca.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state = 1)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

y_predict = knn.predict(X_test)
f1 = f1_score(y_test, y_predict)
print(f'KNN f1 = {f1:.2f}')

svm_classifier = svm.SVC(kernel = 'linear', C = 1, gamma = 'scale')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
_f1 = f1_score(y_test, y_pred)
print(f'SVM f1 = {_f1:.2f}')

KNN f1 = 0.81
SVM f1 = 0.70


经过试验，pca降维并不能提高f1值，本质上降维是对数据的模糊化处理，期间丢失了信息，不能提升准确率是正常的。

我们尝试使用逻辑回归模型，来看一下它的表现。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

raw_data = pd.read_csv('data//fraudulent.csv')
# print(raw_data.shape[0])

for i in range(raw_data.shape[0]):
    w_num = 0
    for j in raw_data.columns:
        if raw_data.loc[i, j] != raw_data.loc[i, j]:
            w_num += 1
    if w_num >= 3:
        raw_data.drop(index = i, inplace = True)
raw_data.reset_index(drop = True, inplace = True)

tmp = [0, 0]
for j in raw_data.columns:
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] == 0 or raw_data.loc[i, j] == 1:
            tmp[int(raw_data.loc[i, j])] += 1
    max_tmp = 0
    if tmp[0] < tmp[1]:
        max_tmp = 1
    tmp = [0, 0]
    # print(max_tmp, end = ' ')
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] != 0 and raw_data.loc[i, j] != 1:
            raw_data.loc[i, j] = max_tmp
y = raw_data['y']
X = raw_data[[_ for _ in raw_data.columns if _ != 'y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state = 1)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

y_predict_knn = knn.predict(X_test)
f1_knn = f1_score(y_test, y_predict_knn)
print(f'KNN f1 = {f1_knn:.2f}')

svm_classifier = svm.SVC(kernel = 'linear', C = 1, gamma = 'scale')
svm_classifier.fit(X_train, y_train)
y_predict_svm = svm_classifier.predict(X_test)
f1_svm = f1_score(y_test, y_predict_svm)
print(f'SVM f1 = {f1_svm:.2f}')

log = LogisticRegression(max_iter = 10000)
log.fit(X_train,y_train)
y_predict_log = log.predict(X_test)
f1_log = f1_score(y_test, y_predict_log)
print(f'Log f1 = {f1_log:.2f}')

KNN f1 = 0.87
SVM f1 = 0.84
Log f1 = 0.84


看起来表现好像一般，我们尝试使用交叉验证和网络搜索来训练模型，以达到更高的拟合度。

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

raw_data = pd.read_csv('data//fraudulent.csv')
# print(raw_data.shape[0])

for i in range(raw_data.shape[0]):
    w_num = 0
    for j in raw_data.columns:
        if raw_data.loc[i, j] != raw_data.loc[i, j]:
            w_num += 1
    if w_num >= 3:
        raw_data.drop(index = i, inplace = True)
raw_data.reset_index(drop = True, inplace = True)

tmp = [0, 0]
for j in raw_data.columns:
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] == 0 or raw_data.loc[i, j] == 1:
            tmp[int(raw_data.loc[i, j])] += 1
    max_tmp = 0
    if tmp[0] < tmp[1]:
        max_tmp = 1
    tmp = [0, 0]
    # print(max_tmp, end = ' ')
    for i in range(raw_data.shape[0]):
        if raw_data.loc[i, j] != 0 and raw_data.loc[i, j] != 1:
            raw_data.loc[i, j] = max_tmp
y = raw_data['y']
X = raw_data[[_ for _ in raw_data.columns if _ != 'y']]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state = 1)
param_grid_knn = {'n_neighbors':[2, 3, 4, 5, 6, 7, 8, 9, 10]}
param_grid_svm = {'C':[0.1, 0.5, 1, 5, 10]}
param_grid_log = {'max_iter':[1000, 5000, 10000, 50000, 100000]}

knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv = 5, scoring = 'f1')
grid_search_knn.fit(X_train, y_train)
knn_best = grid_search_knn.best_estimator_
f1_knn = knn_best.score(X_test, y_test)
print(f'KNN f1 = {f1_knn:.2f}')

svm_classifier = svm.SVC(kernel = 'linear', gamma = 'scale')
grid_search_svm = GridSearchCV(svm_classifier, param_grid_svm, cv = 5, scoring = 'f1')
grid_search_svm.fit(X_train, y_train)
svm_best = grid_search_svm.best_estimator_
f1_svm = svm_best.score(X_test, y_test)
print(f'SVM f1 = {f1_svm:.2f}')

log = LogisticRegression()
grid_search_log = GridSearchCV(log, param_grid_log, cv = 5, scoring = 'f1')
grid_search_log.fit(X_train, y_train)
log_best = grid_search_log.best_estimator_
f1_log = log_best.score(X_test, y_test)
print(f'Log f1 = {f1_log:.2f}')

KNN f1 = 0.91
SVM f1 = 0.90
Log f1 = 0.90


可以看到三个模型的f1都达到了0.9及以上，比起之前的0.85已经相当高了。